<head><h1 align="center">
Food.com Recipe Interactions
</h1></head>  
  
<head><h3 align="center">Recipe Recommender System Modeling</h3></head>

We will develop a **personalized recommender system**. With such systems, there are different options for methods in which to approach recommendations:
  
- The first method is to ask for *explicit* defined ratings from a user regarding the content that they have used/viewed/consumed.  

- Another method is to gather data *implicitly* as the user interacts with the system or service.
  
In this dataset, we have both:  
- **Explicit Data** — the `rating` feature of the **interactions dataset** for each `user`.
- **Implicit Data** — the `x` feature of the **x dataset**.  
  
Technically, I would suppose the `review` feature would fall under explicit data as well, but to use it as such would require *Natural Language Processing*, which we may get involved in later, but not in this notebook.  
  
Additionally with personalized recommender systems, we can center our system around either around suggesting similar *content* (recipes) or suggestions based on other *users of similar preference*. Both systems are succinctly described below:
  
**Content-Based Recommenders**  
> Main Idea: If you like an item, you will also like "similar" items.$_1$  
  
**Collaborative Filtering Systems**  
> Main Idea: If user A likes items 5, 6, 7, and 8 and user B likes items 5, 6, and 7, then it is highly likely that user B will also like item 8.$_1$  
  
We will use `scikit-surprise` for a user-based collaborative filtering model.

#### Setup

In [2]:
%conda install --yes --prefix {sys.prefix} -c conda-forge scikit-surprise

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::nbclassic==0.2.6=pyhd8ed1ab_0
  - conda-forge/linux-64::blaze==0.11.3=py36_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::jupyterlab==3.0.9=pyhd8ed1ab_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/linux-64::bokeh==2.2.3=py36h5fab9bb_0
  - conda-forge/noarch::aiobotocore==1.2.1=pyhd8ed1ab_0
  -

In [5]:
# import libraries
import boto3, re, sys, time, math, csv, json, os, pickle, sagemaker, urllib.request
from os import system
from math import floor
from time import gmtime, strftime
from copy import deepcopy
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cdist
from surprise.similarities import cosine, msd, pearson # for Memory-based Methods (Neighborhood-based)
from surprise.prediction_algorithms import SVD, knns, KNNWithMeans, KNNBasic, KNNBaseline, KNNWithZScore, CoClustering, BaselineOnly, NormalPredictor, NMF, SVDpp, SlopeOne
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split
from surprise import Reader, Dataset, accuracy, dump
                  
from IPython.display import Image                 
from IPython.display import display               

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 200)

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/knn'
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/knn:1',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/knn:1',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/knn:1',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/knn:1',
              'ap-northeast-1': '351501993468.dkr.ecr.ap-northeast-1.amazonaws.com/knn:1',
              'ap-northeast-2': '835164637446.dkr.ecr.ap-northeast-2.amazonaws.com/knn:1',
              'ap-southeast-2': '712309505854.dkr.ecr.ap-southeast-2.amazonaws.com/knn:1'}

my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-2 region. You will use the 404615174143.dkr.ecr.us-east-2.amazonaws.com/knn:1 container for your SageMaker endpoint.


In [6]:
bucket = 'sagemaker-studio-t1ems8mtnoj'
subfolder = ''
s3 = boto3.client('s3')
contents = s3.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
# for f in contents:
#     print(f['Key'])

# Data Import & Preparation

In [7]:
idf = pd.read_csv('../data/RAW_interactions.csv')
idf['date'] = pd.to_datetime(idf.date)
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).
sidf = Dataset.load_from_df(idf[['user_id', 'recipe_id', 'rating']], reader)

In [10]:
rdf = pd.read_csv('../data/RAW_recipes.csv')
idf = pd.read_csv('../data/RAW_interactions.csv')

# pp_rdf = pd.read_csv('../data/PP_recipes.csv')
# pp_idf = pd.read_csv('../data/PP_users.csv')

# ingID = pd.read_pickle('../data/ingr_map.pkl')

# tdf = pd.read_csv('../data/interactions_train.csv')
# vdf = pd.read_csv('../data/interactions_validation.csv')

# """Cleaning: All steps carried over from EDA Notebook"""
rdf.drop(labels=721, inplace = True)

# Cleaning/FE: Creating columns for recipe's respective nutrients
rdf['kcal'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[0])
rdf['fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[1])
rdf['sugar'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[2])
rdf['salt'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[3])
rdf['protein'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[4])
rdf['sat_fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[5])
rdf['carbs'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[6])

# Cleaning: Imputing outlier value to median
rdf['minutes'] = np.where(rdf.minutes == 2147483647,
                         rdf.minutes.median(),
                         rdf.minutes)

idf['date'] = pd.to_datetime(idf.date)

# Model-based Collaborative Filtering with `scikit-Surprise`

In [3]:
idf.columns

Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

### Data Preparation

In [9]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

In [10]:
# The columns must correspond to user id, item id and ratings (in that order).
sidf = Dataset.load_from_df(idf[['user_id', 'recipe_id', 'rating']], reader)

In [11]:
type(sidf)

surprise.dataset.DatasetAutoFolds

### Prediction Algorithm Cross Validation Splits

#### BaselineOnly Algorithm with ALS

Our first model, with 5 cross validation folds and hyperparameter tuning otherwise set to default settings, performed with a predicted rating mean RMSE of **1.2127**. I would be interested in seeing how tweaking the method used by the baseline estimator to Stochastic Gradient Descent instead changes outcomes, as well as changing the number of cross validation folds.

Results:
- mean across folds: **1.2127**  
- std across folds: **0.0034**
- mean fit time across folds: **5.84**
- mean test time across folds: **1.93**

In [12]:
cross_validate(BaselineOnly(bsl_options={'method':'als'}), sidf, cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2074  1.2152  1.2161  1.2100  1.2150  1.2127  0.0034  
MAE (testset)     0.7342  0.7374  0.7389  0.7372  0.7383  0.7372  0.0016  
Fit time          5.24    5.96    6.01    5.92    6.05    5.84    0.30    
Test time         1.92    1.93    1.94    2.44    1.41    1.93    0.33    


{'test_rmse': array([1.20743482, 1.21524522, 1.21607505, 1.21002917, 1.21495625]),
 'test_mae': array([0.7341978 , 0.73737237, 0.7389167 , 0.73718265, 0.73831716]),
 'fit_time': (5.243390083312988,
  5.9565110206604,
  6.009812355041504,
  5.9188947677612305,
  6.051447153091431),
 'test_time': (1.9163506031036377,
  1.9299907684326172,
  1.9351892471313477,
  2.441502809524536,
  1.4138259887695312)}

#### Baseline with Stochastic Gradient Descent

This model performed with the most marginal difference when compared to the previous baseline ALS model. With a mean RMSE across the 5 folds of **1.2126**, it performed *slightly better* (by only **.0001!**). However, the marginal improvement doesn't take fit time in to consideration, with our ALS model performing 63.8% faster.

In [12]:
6.56/10.44

0.6283524904214559

In [22]:
cross_validate(BaselineOnly(bsl_options={'method':'sgd'}), sidf, cv=5, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2110  1.2179  1.2107  1.2116  1.2115  1.2126  0.0027  
MAE (testset)     0.7347  0.7374  0.7345  0.7344  0.7337  0.7349  0.0013  
Fit time          9.78    10.36   10.90   10.66   10.51   10.44   0.38    
Test time         2.51    1.47    1.51    1.49    2.04    1.80    0.41    


{'test_rmse': array([1.21103128, 1.21793577, 1.21072264, 1.21160403, 1.21147109]),
 'test_mae': array([0.73465602, 0.73739129, 0.73451846, 0.7343926 , 0.73368812]),
 'fit_time': (9.78078579902649,
  10.355447053909302,
  10.901717185974121,
  10.656184911727905,
  10.510430335998535),
 'test_time': (2.512019157409668,
  1.472841739654541,
  1.5068244934082031,
  1.4883172512054443,
  2.0438711643218994)}

#### ALS with 25 Cross Validation Folds

As the dataset is so large, I was curious to see how increasing the number of testing folds impacted performance. While difficult to read the output, the model performed better, with a mean RMSE across all folds of **1.2099**. and a higher standard deviation of 0.0070.

In [24]:
cross_validate(BaselineOnly(bsl_options={'method':'als'}), sidf, cv=25, verbose=False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.21121645, 1.21533642, 1.20510059, 1.19654886, 1.20262649,
        1.19608838, 1.21444027, 1.22091298, 1.2036998 , 1.22010393,
        1.20979793, 1.2182443 , 1.20386926, 1.1846379 , 1.19784282,
        1.19744802, 1.20849942, 1.2213187 , 1.21482322, 1.22405952,
        1.21095789, 1.22512724, 1.20504865, 1.22632917, 1.21335891]),
 'test_mae': array([0.73420097, 0.7374428 , 0.73089555, 0.72893016, 0.73054617,
        0.72756061, 0.73797551, 0.74184481, 0.73254516, 0.74045312,
        0.73363478, 0.73811918, 0.73186938, 0.71924968, 0.72957397,
        0.72771911, 0.73403776, 0.74130163, 0.73758994, 0.74146319,
        0.73611061, 0.74367763, 0.73163778, 0.74340695, 0.73610619]),
 'fit_time': (6.769203186035156,
  7.256915330886841,
  7.428053379058838,
  7.269388914108276,
  7.540513515472412,
  7.337223052978516,
  7.693350553512573,
  7.514584302902222,
  7.498729467391968,
  7.537734508514404,
  7.461846828460693,
  7.50420355796814,
  7.5506346225738525,
  7.23

#### CoClustering

While this algorithm produced performance metrics not very far off from our other models, it performed worse all around—mean across folds, variance across folds, and fit time for each fold.

In [ ]:
cross_validate(CoClustering(random_state=1984), sidf, cv=5, verbose=True)

#### SVD

This model performed with a mean RMSE of **1.2201** and a standard deviation of 0.0026 across all folds, but the fit time was *obscenely high* in comparison to the previous models. There are also many hyperparameters that may be tuned. Despite the much longer train time, let's see how hyper parameter tuning with **GridSearchCV** changes performance.  
  
Note that this model is performed with surprise's SVD algorithm's default hyperparameters:  
- `n_factors`: **100**
- `n_epochs`: **20**
- `lr_all`: **0.005** – The learning rate for all parameters.
- `reg_all`: **0.02** – The regularization term for all parameters.

In [42]:
cross_validate(SVD(random_state=1984), sidf, cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2253  1.2232  1.2208  1.2100  1.2214  1.2201  0.0053  
MAE (testset)     0.7413  0.7409  0.7393  0.7344  0.7399  0.7391  0.0025  
Fit time          55.76   55.60   55.61   55.55   55.14   55.53   0.21    
Test time         2.25    2.23    2.36    2.22    2.16    2.25    0.06    


{'test_rmse': array([1.22526615, 1.22324572, 1.22075873, 1.20997109, 1.22137993]),
 'test_mae': array([0.74128305, 0.74086325, 0.7393137 , 0.73437507, 0.73991007]),
 'fit_time': (55.75669455528259,
  55.60287833213806,
  55.61086654663086,
  55.54658079147339,
  55.135498046875),
 'test_time': (2.2547242641448975,
  2.233457088470459,
  2.358004093170166,
  2.2175631523132324,
  2.1614022254943848)}

## Hyperparameter Tuning

After many attempts with surprises' GridSearchCV, attempts have been unsuccessful in getting it to function appropriately for evaluating hyperparameter tuning of the few options available in the BaselineOnly algorithm. I will tune these manually, one hyperparameter at a time, holding all else equal.

### Baseline ALS

**Default Hyperparameters for Baseline ALS:**  
>     bsl_options = {
                'method': 'als', # setting method to Alternating Least Squares
                'n_epochs': 10, # number of iterations of the ALS procedure.
                'reg_u': 15, # user regularization. Corresponding to 𝜆3 in Koren:2010
                'reg_i': 10 # item similarity regularization. Corresponding to 𝜆2 in Koren:2010
}

In [3]:
bsl_options = {'method':'als',
               'n_epochs':[5, 10, 15, 20],  
               'reg_u':[10, 15, 20], 
               'reg_i':[5, 10, 15]}

#### Tuning Epochs, All Else Same

Both ALS models tuned with 5 epochs and 15 epochs performed exactly as well as each other on RMSE, and .0001 *worse* than the model with 10 epochs.  
  
It appears that tuning the epochs (though tuning is modest) is having little to no effect on the performance of the model on the evaluation metric. Despite this, I will opt to use 5 epochs, as fit time is significantly better.

**epochs = 10, reg_u = 15, reg_i = 10 (Defaults):**
- Mean RMSE across folds: **1.2099**  
- Std across folds: **0.0040**
- Mean fit time across folds: **9.24**
- Mean test time across folds: **0.37**

**5 epochs:**
- Mean RMSE: **1.2100**
- Std: **0.0079**
- Mean Fit Time: **4.71**
- Mean Test Time: **0.39**

**15 epochs:**
- Mean RMSE: **1.2100**
- Std: **0.0090**
- Mean Fit Time: **11.05**
- Mean Test Time: **0.39**

In [8]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 5,
                                           'reg_u': 15,
                                           'reg_i': 10}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.2156831 , 1.20531208, 1.21442359, 1.21042687, 1.21060443,
        1.20296688, 1.22064749, 1.20564088, 1.21272104, 1.19679772,
        1.20170394, 1.21091501, 1.22864905, 1.19975855, 1.20265508,
        1.20824238, 1.20823335, 1.20732816, 1.22056131, 1.21127556,
        1.20497221, 1.20730944, 1.20761959, 1.2292085 , 1.20542145]),
 'test_mae': array([0.73681407, 0.73080775, 0.73510277, 0.73130956, 0.73554578,
        0.73125715, 0.73973386, 0.73382419, 0.73450594, 0.72822356,
        0.73044812, 0.73514672, 0.74464898, 0.73164037, 0.73005156,
        0.7329784 , 0.73264626, 0.73446754, 0.74102113, 0.73682453,
        0.73095641, 0.73413787, 0.73498359, 0.74602854, 0.73228744]),
 'fit_time': (3.71569561958313,
  4.6102776527404785,
  4.569671154022217,
  4.591548681259155,
  4.5800697803497314,
  4.667249441146851,
  4.749887466430664,
  4.808042287826538,
  4.748213052749634,
  4.70083212852478,
  4.602090835571289,
  4.6301469802856445,
  4.658377170562744,
  4.7

In [9]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 15,
                                           'reg_u': 15,
                                           'reg_i': 10}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.21123854, 1.21410024, 1.20764912, 1.2132602 , 1.19942733,
        1.21843453, 1.1984201 , 1.19744475, 1.20403879, 1.21975023,
        1.20884325, 1.19338084, 1.20493593, 1.21088942, 1.20246804,
        1.1988338 , 1.20540815, 1.20486006, 1.21158384, 1.22306126,
        1.22065923, 1.22368543, 1.20961307, 1.22559585, 1.22124651]),
 'test_mae': array([0.73532118, 0.73769301, 0.73409756, 0.73794902, 0.73087512,
        0.73837743, 0.72774927, 0.72835429, 0.73192807, 0.73687361,
        0.7350756 , 0.72703858, 0.73255866, 0.73308677, 0.73074432,
        0.72789013, 0.73376061, 0.73090334, 0.73560363, 0.74047395,
        0.74203062, 0.74182881, 0.73379024, 0.74350172, 0.74098366]),
 'fit_time': (10.45699429512024,
  10.967830896377563,
  10.76924467086792,
  11.436404466629028,
  11.500694274902344,
  11.449135303497314,
  10.468889713287354,
  11.055140018463135,
  11.904171466827393,
  11.338705062866211,
  10.709330558776855,
  11.2050199508667,
  10.78197956085205

#### Tuning User Regularization, All Else Same

Lowering user regularization marginally improved RMSE, while raising u_reg performed worse.  
  
Note: Fit time for default was run on a machine without a GPU, and is the cause of the near halving of fit time.

**reg_u = 15 (Default)**, epochs = 5, reg_i = 10:
- Mean RMSE: **1.2100**
- Std: **0.0079**
- Mean Fit Time: **4.71**
- Mean Test Time: **0.39**

**reg_u = 10:**
- Mean RMSE: **1.2082**
- Std: **0.0069**
- Mean Fit Time: **4.72**
- Mean Test Time: **0.41**

**reg_u = 20:**
- Mean RMSE: **1.2115**
- Std: **0.0061**
- Mean Fit Time: **4.71**
- Mean Test Time: **0.31**

In [10]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 5,
                                           'reg_u': 10,
                                           'reg_i': 10}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.20714884, 1.20121438, 1.21058119, 1.21583974, 1.20684211,
        1.20323102, 1.23149948, 1.20058317, 1.20426017, 1.20460216,
        1.2166258 , 1.20788559, 1.20310975, 1.20290037, 1.21148016,
        1.20749178, 1.20880296, 1.21442186, 1.2159409 , 1.20694339,
        1.19870733, 1.20885092, 1.20897566, 1.19853503, 1.20769587]),
 'test_mae': array([0.73061982, 0.72694581, 0.73067118, 0.73219393, 0.73016403,
        0.72820726, 0.74472718, 0.72515984, 0.72688582, 0.72956497,
        0.73263705, 0.7302993 , 0.72855374, 0.72673863, 0.73183739,
        0.73115183, 0.73163261, 0.73339941, 0.73394131, 0.73087587,
        0.72515561, 0.72984768, 0.73083145, 0.72600945, 0.731461  ]),
 'fit_time': (3.8925015926361084,
  4.476735830307007,
  4.6840126514434814,
  4.698370695114136,
  4.714624404907227,
  4.771652460098267,
  6.188734292984009,
  4.67169451713562,
  4.717334270477295,
  4.6708824634552,
  4.662247657775879,
  4.565034627914429,
  4.649930000305176,
  4.639

In [11]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 5,
                                           'reg_u': 20,
                                           'reg_i': 10}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.20827877, 1.21075024, 1.22295025, 1.21289304, 1.21133508,
        1.20321136, 1.20582665, 1.21788514, 1.20931708, 1.20710198,
        1.21647564, 1.21053922, 1.20364794, 1.21765005, 1.20877544,
        1.21562359, 1.21337561, 1.20873889, 1.19995432, 1.22377187,
        1.21609477, 1.20803192, 1.22119856, 1.20755259, 1.20553626]),
 'test_mae': array([0.73657844, 0.7375299 , 0.74332997, 0.73911289, 0.73740842,
        0.72999135, 0.7333319 , 0.73968649, 0.73657516, 0.73431937,
        0.74120999, 0.73626872, 0.73523042, 0.74147685, 0.73810945,
        0.74100639, 0.73739694, 0.73800694, 0.73226175, 0.7456189 ,
        0.74146456, 0.73541301, 0.74478187, 0.7348043 , 0.73396115]),
 'fit_time': (3.9059348106384277,
  4.595790147781372,
  4.858780145645142,
  4.641463279724121,
  4.6920177936553955,
  4.792471885681152,
  4.599896669387817,
  4.702898025512695,
  4.744200706481934,
  4.804708480834961,
  4.720059633255005,
  4.702065467834473,
  4.7934465408325195,
  4

#### Tuning Item Regularization, All Else Same

Tuning item regularization did not produce a better performing model than default setting for reg_i.

**reg_i = 10 (Default), n_epochs = 5, reg_u = 10 — BEST MODEL:**
- Mean RMSE: **1.2082**
- Std: **0.0069**
- Mean Fit Time: **4.72**
- Mean Test Time: **0.41**

**reg_i = 5:**
- Mean RMSE: **1.2097**
- Std: **0.0071**
- Mean Fit Time: **4.69**
- Mean Test Time: **0.39**

**reg_i = 15:**
- Mean RMSE: **1.2085**
- Std: **0.0090**
- Mean Fit Time: **4.65**
- Mean Test Time: **0.44**

In [13]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 5,
                                           'reg_u': 10,
                                           'reg_i': 5}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.20011225, 1.21610688, 1.1941272 , 1.21277844, 1.21296482,
        1.20585891, 1.21229227, 1.21364301, 1.20326473, 1.20861119,
        1.20681514, 1.21308355, 1.20550493, 1.21699762, 1.19726467,
        1.2191143 , 1.21269778, 1.21553359, 1.21223593, 1.22406491,
        1.20241989, 1.21343879, 1.21457296, 1.20845686, 1.20039505]),
 'test_mae': array([0.72611451, 0.73508491, 0.72407678, 0.73424717, 0.73177381,
        0.73122741, 0.73323214, 0.73641432, 0.72688209, 0.73315378,
        0.72895522, 0.73178252, 0.7292847 , 0.73616064, 0.72555242,
        0.7354149 , 0.73321171, 0.73387431, 0.73471776, 0.73976546,
        0.72779369, 0.73378704, 0.73272884, 0.73190836, 0.72794007]),
 'fit_time': (3.8653292655944824,
  4.699767112731934,
  4.765350103378296,
  4.76214075088501,
  4.738460540771484,
  5.003239870071411,
  4.841422080993652,
  4.622073173522949,
  4.732499122619629,
  4.828778982162476,
  4.582040309906006,
  4.652557849884033,
  4.698713779449463,
  4.58

In [14]:
cross_validate(BaselineOnly(bsl_options = {'method':'als',
                                           'n_epochs': 5,
                                           'reg_u': 10,
                                           'reg_i': 15}), sidf, cv=25, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.22697409, 1.22286124, 1.21090211, 1.19935455, 1.20868231,
        1.20090302, 1.22033005, 1.20610609, 1.20925004, 1.21179948,
        1.203818  , 1.23352745, 1.20997696, 1.21250823, 1.20138467,
        1.19768687, 1.1993582 , 1.20614894, 1.20365172, 1.19588711,
        1.20538913, 1.20296167, 1.20398047, 1.21021566, 1.20909681]),
 'test_mae': array([0.73867667, 0.73910186, 0.73259891, 0.72475181, 0.73219916,
        0.72513856, 0.73664811, 0.73038966, 0.73161902, 0.73132552,
        0.72716801, 0.74496738, 0.73165702, 0.72997128, 0.72608674,
        0.72483558, 0.72576896, 0.73012536, 0.72665199, 0.72302526,
        0.72947319, 0.72851314, 0.72972181, 0.7311861 , 0.72955809]),
 'fit_time': (3.7923097610473633,
  4.43336296081543,
  4.768854141235352,
  4.736692190170288,
  4.692785739898682,
  4.555355072021484,
  4.6628875732421875,
  4.6176159381866455,
  4.524052619934082,
  4.766663074493408,
  4.8947296142578125,
  4.5377724170684814,
  4.644901990890503,
  

### Baseline SGD

After training several models with tuning hyperparameters individually, keeping others the same per iterations, was unable to produce a better model than BaselineOnly SGD with default parameters.

**Default Hyperparameters for Baseline SGD:**  
>     bsl_options = {
                'method': 'sgd', # setting method to Stochastic Gradient Descent
                'n_epochs': 20, # number of epochs
                'reg': .02, # regularization parameter of the cost function that is optimized, 
>                             # corresponding to 𝜆1 in Koren:2010  
                'learning_rate': .005 # learning rate of SGD. Corresponding to 𝛾 in Koren:2010
}

#### Tuning Epochs, All Else Same

Tuning number of epochs resulted in no better performance than Baseline SGD with default hyperparameters.

**Results with Default bsl_options — sgd, 20 epochs:**
- mean across folds: **1.2095**  
- std across folds: **0.0066**
- mean fit time across folds: **11.80**
- mean test time across folds: **0.39**

**15 epochs:**
- Mean RMSE: **1.2102**
- Std: **0.0076**
- Mean Fit Time: **9.35**
- Mean Test Time: **0.37**

**25 epochs:**
- Mean RMSE: **1.2096**
- Std: **0.0070**
- Mean Fit Time: **14.38**
- Mean Test Time: **0.37**

In [ ]:
bsl_options = {'method':'sgd',
               'n_epochs':[10, 15, 20, 25, 30], 
               'reg':[.008, .015, .2, .25, .3], 
               'learning_rate':[.003, .005, .01, .015, .025]}

In [ ]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 15,
                                           'reg': .02,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.21407101, 1.218625  , 1.2180732 , 1.19782818, 1.20639959,
        1.22361802, 1.21712643, 1.20563144, 1.22039735, 1.20708831,
        1.2089327 , 1.20719348, 1.20034672, 1.20899045, 1.20990721,
        1.21018586, 1.22291804, 1.21166   , 1.19889312, 1.21243516,
        1.20336713, 1.20011389, 1.20385825, 1.22220494, 1.20439464]),
 'test_mae': array([0.73868325, 0.73988448, 0.73893291, 0.73181645, 0.73393651,
        0.74337195, 0.74056963, 0.73410075, 0.74391567, 0.73345754,
        0.73567347, 0.73266845, 0.73057977, 0.73629699, 0.73807876,
        0.73409698, 0.74179626, 0.73569477, 0.72836689, 0.7353866 ,
        0.73140085, 0.73088171, 0.73346588, 0.74084528, 0.73130563]),
 'fit_time': (8.517118215560913,
  9.582951307296753,
  10.000103235244751,
  9.673792600631714,
  9.707358598709106,
  9.674190521240234,
  9.018184900283813,
  9.393586158752441,
  9.276110410690308,
  9.419939517974854,
  9.182628393173218,
  9.185728549957275,
  11.054856300354004,
  9.

In [17]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 25,
                                           'reg': .02,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.19985546, 1.21479142, 1.22521905, 1.21416169, 1.22714511,
        1.20202305, 1.19952615, 1.2112846 , 1.21510649, 1.21047674,
        1.20840966, 1.20420255, 1.21512381, 1.20752626, 1.20790597,
        1.20420335, 1.21784529, 1.2100457 , 1.20239103, 1.20755282,
        1.20020422, 1.20921819, 1.20640525, 1.21033077, 1.20969671]),
 'test_mae': array([0.72950505, 0.73280373, 0.74088326, 0.73392583, 0.74255428,
        0.72882203, 0.7284359 , 0.73372109, 0.73678396, 0.73368503,
        0.73340829, 0.73152668, 0.7357676 , 0.7323201 , 0.72858883,
        0.728679  , 0.73831002, 0.73211047, 0.73130264, 0.73213878,
        0.72660472, 0.73707812, 0.73054503, 0.73415075, 0.73492161]),
 'fit_time': (13.608980178833008,
  14.193682432174683,
  14.202935934066772,
  13.878445863723755,
  14.522101640701294,
  14.22553014755249,
  14.146848201751709,
  14.39509892463684,
  14.300932884216309,
  14.216705083847046,
  13.940992832183838,
  14.465631008148193,
  13.676266908645

#### Tuning Regularization, All Else Same

Tuning of regularization resulted in no better performance than Baseline SGD with default hyperparameters.

**Results with Default bsl_options — sgd, 20 epochs, .02 reg:**
- mean across folds: **1.2095**  
- std across folds: **0.0066**
- mean fit time across folds: **11.80**
- mean test time across folds: **0.39**

**reg = .003:**
- Mean RMSE: **1.2097**
- Std: **0.0076**
- Mean Fit Time: **12.22**
- Mean Test Time: **0.44**

**reg = .01:**
- Mean RMSE: **1.2096**
- Std: **0.0070**
- Mean Fit Time: **12.12**
- Mean Test Time: **0.47**

**reg = .035:**
- Mean RMSE: **1.2095**
- Std: **0.0063**
- Mean Fit Time: **12.16**
- Mean Test Time: **0.51**

**reg = .1:**
- Mean RMSE: **1.2095**
- Std: **0.0090**
- Mean Fit Time: **11.72**
- Mean Test Time: **0.37**

In [ ]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .003,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.21103162, 1.20676699, 1.21215612, 1.19995927, 1.22104996,
        1.20983895, 1.20804236, 1.21267105, 1.2076057 , 1.21265158,
        1.21646886, 1.20013987, 1.21367468, 1.21418666, 1.20983461,
        1.21541414, 1.20204813, 1.19667458, 1.22119627, 1.20377912,
        1.21399696, 1.21498696, 1.20727899, 1.20366579, 1.20672754]),
 'test_mae': array([0.73308217, 0.73096642, 0.73621393, 0.72873047, 0.74185925,
        0.73372914, 0.7331411 , 0.73469543, 0.73122074, 0.7328748 ,
        0.73506291, 0.72685496, 0.73481657, 0.73586421, 0.73264048,
        0.73397197, 0.73151578, 0.72673064, 0.73576039, 0.72955246,
        0.73458108, 0.7366578 , 0.7318989 , 0.73026221, 0.7327711 ]),
 'fit_time': (10.98929500579834,
  12.48154067993164,
  11.980997800827026,
  12.992700576782227,
  11.95482349395752,
  12.123815536499023,
  12.352720022201538,
  12.146445274353027,
  11.907366037368774,
  12.529923677444458,
  12.30427622795105,
  12.583238363265991,
  11.55731439590454

In [19]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .01,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.21344092, 1.19454581, 1.212108  , 1.21615567, 1.21842737,
        1.20803286, 1.20982635, 1.21588044, 1.21603051, 1.21294695,
        1.2082264 , 1.21444881, 1.21059673, 1.21742175, 1.20133835,
        1.21039627, 1.19469589, 1.2201993 , 1.20777673, 1.20554843,
        1.19615831, 1.20986923, 1.2126634 , 1.21122739, 1.20256723]),
 'test_mae': array([0.73425972, 0.72759958, 0.73636775, 0.73647889, 0.73835038,
        0.73169786, 0.73386157, 0.73511632, 0.73563942, 0.73428002,
        0.73268958, 0.73424425, 0.73405546, 0.73680487, 0.72602678,
        0.73388816, 0.72353798, 0.73649612, 0.73265558, 0.73170171,
        0.72485465, 0.73325653, 0.73466893, 0.73406423, 0.72890303]),
 'fit_time': (11.12335729598999,
  11.929053783416748,
  11.703220844268799,
  12.292922258377075,
  12.141081094741821,
  12.219889402389526,
  12.184540033340454,
  12.470080375671387,
  12.279286861419678,
  11.571953058242798,
  11.75516390800476,
  12.10994839668274,
  12.3964395523071

In [20]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .035,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.20322451, 1.20743656, 1.20828749, 1.20841954, 1.22097372,
        1.19767641, 1.20104895, 1.20548254, 1.2086423 , 1.22469781,
        1.21033804, 1.20563971, 1.21301865, 1.21398011, 1.21232144,
        1.21163918, 1.2031189 , 1.20910453, 1.2157378 , 1.20800039,
        1.20703494, 1.21781891, 1.20105655, 1.21775926, 1.2062671 ]),
 'test_mae': array([0.73133902, 0.73112315, 0.73564523, 0.73502773, 0.7418092 ,
        0.72931906, 0.73061178, 0.73270918, 0.7351611 , 0.74096242,
        0.73582377, 0.73287481, 0.73582422, 0.7364383 , 0.73631289,
        0.73477692, 0.73214389, 0.73502879, 0.73732417, 0.73735609,
        0.73222487, 0.73995069, 0.72921831, 0.74078815, 0.73340776]),
 'fit_time': (11.230709791183472,
  12.198558330535889,
  12.522706031799316,
  11.880070924758911,
  12.59970498085022,
  13.08237886428833,
  12.015311479568481,
  12.300896167755127,
  12.218340873718262,
  12.1530282497406,
  11.865094184875488,
  12.203782796859741,
  12.38823628425598

In [ ]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .1,
                                           'learning_rate': .005}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.21579448, 1.21189506, 1.22123904, 1.21872088, 1.21325073,
        1.21286902, 1.20589957, 1.19952524, 1.21480653, 1.22183348,
        1.22069064, 1.20388905, 1.20470136, 1.19401548, 1.20359753,
        1.2340159 , 1.20091863, 1.19997939, 1.21103237, 1.20668259,
        1.20207483, 1.20523629, 1.20999524, 1.19808487, 1.20559938]),
 'test_mae': array([0.7408256 , 0.74025019, 0.74583416, 0.74219813, 0.74042644,
        0.74086106, 0.73910079, 0.73410822, 0.74407995, 0.74762788,
        0.74303976, 0.73787765, 0.73696066, 0.73059371, 0.73589031,
        0.75333169, 0.73534439, 0.7352674 , 0.74078447, 0.73704165,
        0.73593744, 0.73698068, 0.73922698, 0.73236818, 0.73693003]),
 'fit_time': (10.546465873718262,
  11.586823225021362,
  11.604610919952393,
  12.4130380153656,
  11.538708209991455,
  11.767252206802368,
  11.930396556854248,
  11.919605255126953,
  11.704459190368652,
  11.41018795967102,
  12.167648077011108,
  11.826277732849121,
  11.6658818721771

#### Tuning Learning Rate, All Else Same

Tuning learning rate resulted in no better performance than Baseline SGD with default hyperparameters.

**Results with Default bsl_options — sgd, 20 epochs, .02 reg, .005 LR:**
- mean across folds: **1.2095**  
- std across folds: **0.0066**
- mean fit time across folds: **11.80**
- mean test time across folds: **0.39**

**LR = .00015:**
- Mean RMSE: **1.2372**
- Std: **0.0075**
- Mean Fit Time: **11.65**
- Mean Test Time: **0.41**

**LR = .025:**
- Mean RMSE: **1.2295**
- Std: **0.0072**
- Mean Fit Time: **11.54**
- Mean Test Time: **0.32**

In [22]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .02,
                                           'learning_rate': .00015}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.24098833, 1.24495666, 1.23307736, 1.23809773, 1.24316453,
        1.22973449, 1.24048417, 1.24162502, 1.23119878, 1.22725598,
        1.2407301 , 1.24085768, 1.25682717, 1.24237145, 1.24403541,
        1.24110176, 1.23695429, 1.23025278, 1.23902726, 1.23644369,
        1.23831601, 1.22340118, 1.23719541, 1.23149256, 1.22031463]),
 'test_mae': array([0.77826692, 0.77881627, 0.77393949, 0.77635113, 0.77876119,
        0.77155299, 0.77887529, 0.77805444, 0.77321206, 0.76920757,
        0.77776311, 0.7776644 , 0.78596801, 0.77890011, 0.77870606,
        0.7785013 , 0.77289368, 0.77191298, 0.77687806, 0.77556484,
        0.7769471 , 0.77027307, 0.77636501, 0.77361111, 0.76823686]),
 'fit_time': (10.814207315444946,
  11.835895776748657,
  11.69747543334961,
  11.491311073303223,
  11.773432731628418,
  11.94485592842102,
  11.917098760604858,
  11.852519512176514,
  11.678627729415894,
  11.810724258422852,
  11.807735919952393,
  11.50357985496521,
  11.5540003776550

In [23]:
cross_validate(BaselineOnly(bsl_options = {'method':'sgd',
                                           'n_epochs': 20,
                                           'reg': .02,
                                           'learning_rate': .025}), sidf, cv=25, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Evaluating RMSE, MAE of algorithm BaselineOnly on 25 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18

{'test_rmse': array([1.23567146, 1.24241263, 1.22816815, 1.22686586, 1.22629292,
        1.22847673, 1.23089527, 1.24555107, 1.22163743, 1.22264371,
        1.23171067, 1.23336354, 1.23106014, 1.22164156, 1.22810059,
        1.24044674, 1.22755975, 1.23404351, 1.23003274, 1.21945979,
        1.21462815, 1.22319115, 1.22439443, 1.23782207, 1.23250173]),
 'test_mae': array([0.74953279, 0.75274975, 0.74532191, 0.74515823, 0.74496418,
        0.74339446, 0.74539375, 0.75465291, 0.74220374, 0.7430333 ,
        0.74552355, 0.74920752, 0.7433502 , 0.74359563, 0.74776935,
        0.75215029, 0.74481759, 0.74733708, 0.7474043 , 0.74112268,
        0.73801361, 0.7424784 , 0.73837651, 0.74867344, 0.7492429 ]),
 'fit_time': (10.694491386413574,
  11.29405689239502,
  11.928758144378662,
  11.455387353897095,
  11.243992805480957,
  11.307096719741821,
  11.77086353302002,
  12.105730772018433,
  11.13573169708252,
  11.402637004852295,
  11.852785587310791,
  11.763862371444702,
  11.5914621353149

### Grid Search Cross Validation Split

#### ALS (Failed)

In [ ]:
param_grid = {'bsl_options':{'method':'als',
              'n_epochs':[5, 10, 15, 20],  
              'reg_u':[10, 15, 20], 
              'reg_i':[5, 10, 15]}}
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=25)
gs.fit(sidf)
params = gs.best_params['rmse']
gs_als = BaselineOnly(n_factors=params['n_factors'], n_epochs=params['n_epochs'], lr_all=params['lr_all'], reg_all=params['reg_all'])

#### Baseline SGD (Failed)

In [ ]:
param_grid = {'bsl_options':{'method':'sgd',
                             'n_epochs':[10, 15, 20, 25, 30], 
                             'reg':[.008, .015, .2, .25, .3], 
                             'learning_rate':[.003, .005, .01, .015, .025]}}
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=5)
gs.fit(sidf)
params = gs.best_params['rmse']
gs_als = BaselineOnly(n_factors=params['n_factors'], n_epochs=params['n_epochs'], lr_all=params['lr_all'], reg_all=params['reg_all'], verbose=True)

#### SVD

Here I performed GridSearchCV the SVD function to see if it can be optimized further.  
After performing grid search cross validation, the optimum hyperparameters for SVD resulted in the following:
- `'n_factors'`: **20** 
- `'n_epochs'`: **10** 
- `'lr_all'`: **0.01** 
- `'reg_all'`: **0.05**  
  
The with optimized hyperparameters, SVD did <u>**NOT perform better than my baseline models**</u>, with an RMSE of <u>**1.2142**</u>. Note that the optimum `n_factors` and `reg_all` were the minumum values instructed to for the grid search to evaluate, while the `n_epochs` and `lr_all` were the maximum. To evaluate further, I could perform a grid search with hyperparameters adjacent to those above.

In [13]:
param_grid = {'n_factors':[20, 50],'n_epochs':[5, 10],  'lr_all':[0.005,0.01], 'reg_all':[0.002,0.05]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gs.fit(sidf)
params = gs.best_params['rmse']
gs_svd = SVD(n_factors=params['n_factors'], n_epochs=params['n_epochs'], lr_all=params['lr_all'], reg_all=params['reg_all'], verbose=True, random_state=1984)

In [41]:
cross_validate(gs_svd, sidf, cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2144  1.2081  1.2217  1.2184  1.2083  1.2142  0.0054  
MAE (testset)     0.7393  0.7367  0.7427  0.7424  0.7375  0.7397  0.0025  
Fit time          11.14   11.10   11.17   11.25   10.97   11.13   0.09    
Test time         1.98    2.05    2.03    2.05    1.96    2.02    0.04    


{'test_rmse': array([1.21435208, 1.20805289, 1.2216954 , 1.21839995, 1.20829103]),
 'test_mae': array([0.7393403 , 0.73665142, 0.74265809, 0.7424117 , 0.73749491]),
 'fit_time': (11.142059564590454,
  11.095774412155151,
  11.170443296432495,
  11.254150390625,
  10.971884965896606),
 'test_time': (1.9834518432617188,
  2.0528981685638428,
  2.032770872116089,
  2.0463125705718994,
  1.9609873294830322)}

In [21]:
params

{'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.01, 'reg_all': 0.05}

In [28]:
type(gs_svd)

surprise.prediction_algorithms.matrix_factorization.SVD

# Model Serialization

#### Using Surprise's `dump` module:

In [38]:
trainset = sidf.build_full_trainset()

algo = gs_svd
algo.fit(trainset)

# Compute predictions of the 'original' algorithm.
predictions = algo.test(trainset.build_testset())

# Here we use scikit-surprise's dump module to save the optimized algorithm state.
file_name = 'models/serialized_optimum_svd_model'
dump.dump(file_name, algo=algo)
_, loaded_algo = dump.load(file_name) # ... and here we reload the file...

# and now we ensure that the algo is still the same by checking the predictions.
predictions_loaded_algo = loaded_algo.test(trainset.build_testset())
assert predictions == predictions_loaded_algo
print('Predictions are the same')

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


#### Using Pickle

In [ ]:
pickle_out = open("optimized_svd_model.pickle","wb")
pickle.dump(gs_svd, pickle_out)
pickle_out.close()

# Model Parallelism

While `scikit-surprise` is not inherently equipped to run computations on cloud distributed network clusters, AWS allows for model parallelism. 

## Collaborative Filtering Models

**KNN with Means**  
- First I will run KNN Means on user-user similarity, and will evaluate item-item similarity predictions as well. 
- I use **Pearson** correlation coefficient as my siilarity metric, as it best accounts for ratings of `0` not actually representing values less than ratings of `1`. 
- Additionally, I increased the number for nearest `k` to 350, from the default of 40. With a dataset so large, it would be a shame not to consider many users. I could increase this further in later iterations.  
- Due to memory restrictions, I've reduced the numebr for `k` to see if we can get the algorithm to function.

In [ ]:
cross_validate(KNNWithMeans(k=10, sim_options={'name':'pearson', 'user_based':'True'}), sidf_zero, measures='rmse', cv=5, verbose=True)

In [ ]:
cross_validate(KNNWithMeans(k=10, sim_options={'name':'pearson', 'user_based':'True'}), sidf, measures='rmse', cv=5, verbose=True)

In [ ]:
# param_grid = {'n_factors':[20, 50],'n_epochs':[5, 10],  'lr_all':[0.005,0.01], 'reg_all':[0.002,0.05]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
# gs.fit(sidf)
# params = gs.best_params['rmse']
# gs_svd = SVD(n_factors=params['n_factors'], n_epochs=params['n_epochs'], lr_all=params['lr_all'], reg_all=params['reg_all'], verbose=True, random_state=1984)

**KNNWithMeans Parameters:**
- n_factors: **100** (Default) – The number of factors. 
- `n_epochs`: **20** (Default) – The number of iteration of the SGD procedure. 
- `biased`: **True** (Default) (bool) – Whether to use baselines (or biases). See note above. 
- `init_mean`: **0** (Default) – The mean of the normal distribution for factor vectors initialization. 
- `init_std_dev`: (Default) – The standard deviation of the normal distribution for factor vectors initialization. 
- `lr_all`: **0.005** (Default) – The learning rate for all parameters.
- `reg_all`: **0.02** (Default) – The regularization term for all parameters. 
- `lr_bu`: **None** (Default) – The learning rate for 𝑏𝑢. Takes precedence over lr_all if set. 
- `lr_bi`: **None** (Default) – The learning rate for 𝑏𝑖. Takes precedence over lr_all if set. 
- `lr_pu`: **None** (Default) – The learning rate for 𝑝𝑢. Takes precedence over lr_all if set. 
- `lr_qi`: **None** (Default) – The learning rate for 𝑞𝑖. Takes precedence over lr_all if set. 
- `reg_bu`: **None** (Default) – The regularization term for 𝑏𝑢. Takes precedence over reg_all if set. 
- `reg_bi`: **None** (Default) – The regularization term for 𝑏𝑖. Takes precedence over reg_all if set. 
- `reg_pu`: **None** (Default) – The regularization term for 𝑝𝑢. Takes precedence over reg_all if set. 
- `reg_qi`: **None** (Default) – The regularization term for 𝑞𝑖. Takes precedence over reg_all if set. 
- `random_state`: **None** (Default) (int, RandomState instance from numpy, or None) – Determines the RNG that will be used for initialization. If int, random_state will be used as a seed for a new RNG. This is useful to get the same initialization over multiple calls to fit(). If RandomState instance, this same instance is used as RNG. If None, the current RNG from numpy is used. Default is None.
- `verbose`: **False** (Default) – If True, prints the current epoch. 

In [ ]:
cross_validate(KNNWithMeans(k=100, sim_options={'name':'pearson', 'user_based':'True'}), sidf, measures='rmse', cv=5, verbose=True)

In [13]:
cross_validate(KNNWithMeans(k=350, sim_options={'name':'pearson', 'user_based':'True'}), sidf, measures='rmse', cv=50, verbose=True)

Computing the pearson similarity matrix...


MemoryError: Unable to allocate 371. GiB for an array with shape (223246, 223246) and data type int64

In [17]:
# param_grid = {'n_factors':[50,100,150],'n_epochs':[20,30],  'lr_all':[0.005,0.01],'reg_all':[0.02,0.1]}
# gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, joblib_verbose=1)
# gs.fit(sidf)
# params = gs.best_params['rmse']
# svdtuned = SVD(n_factors=params['n_factors'], n_epochs=params['n_epochs'],lr_all=params['lr_all'], reg_all=params['reg_all'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [47]:
print('Number of users: ', sidf.df.user_id.value_counts().index.shape[0], '\n')
print('Number of recipes: ', sidf.df.recipe_id.value_counts().index.shape[0], '\n')


Number of users:  226570 

Number of recipes:  231637 



In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

# svd = SVD(n_factors=100, n_epochs=10, lr_all=0.005, reg_all=0.4)
# svd.fit(trainset)
# predictions = svd.test(testset)
# print(accuracy.rmse(predictions))

## Memory-based

In [49]:
trainset, testset = train_test_split(sidf, test_size = 0.2, random_state = 1984)

In [50]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  192093 

Number of items:  211233 



In [53]:
sim_cos = {'name':'cosine', 'user_based':True}

In [54]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 332. GiB for an array with shape (211233, 211233) and data type float64

### Collaborative Filtering — User-based Recommender System

>- Collaborative Filtering (CF) is currently the most widely used approach to build recommendation systems  
>- The key idea behind CF is that similar users have similar interests and that a user generally likes items that are similar to other items they like  
>- CF is filling an "empty cell" in the utility matrix based on the similarity between users or item. Matrix factorization or decomposition can help us solve this problem by determining what the overall "topics" are when a matrix is factored
  
Note: We will likely need to bring this notebook over to DataBricks or AWS.  
  
We can use **cosine similarity** between users, or **Pearson Correlation Coefficient**. Different metrics will offer different results, but I don't see why we couldn't offer multiple recommendations using different metrics.

**Note: Don't forget to downplay zeros as they are gaps, not values representative of sentiment!**

### SVD — Singular Value (Matrix) Decomposition

Notes:  
- eigendecompositions require square matrices
- for SVD, we can create square matrices by taking the dot product of a matrix and its transpose
- we can use SVD to draw vectors in a new space to capture as much of the variance in our data as possible

~~Below we use the raw interactions dataset to create the rating matrix `A`, with rows representing recipes and columns representing users.~~  
NVM LOOKS LIKE WE'RE GONNA NEED DISTRIBUTED COMPUTING FOR EVEN A SIMPLE SVD



In [14]:
A = np.ndarray(
    shape=(np.max(idf.recipe_id.values), np.max(idf.index.values)),
    dtype=np.uint8)

A[idf.recipe_id.values-1, idf.index.values-1] = idf.rating.values

MemoryError: Unable to allocate 567. GiB for an array with shape (537716, 1132366) and data type uint8

### ALS — Alternating Lease Squares Matrix Decomposition

- plug in guess values for P and Q
- hold the values of one constant, then use the values for R and the non-constant to find the optimum values for.
- repeat for the other

> "When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

For collaborative ALS, we will want our data to be shaped something like below, where the column numbers represent different recipes.  

Recipe->| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |...| n  
:-------|:--|---|---|---|---|---|---|---|---|---|---|---|---|---|---:  
user_01 | 0 | 0 | 5 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_02 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_03 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 5 | 0 | 0  
user_04 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_05 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_06 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_07 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 0  
  ...   
user_n  | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  


# Evaluation

# Reference

## Reading in Big Data from S3

If the above read_csv statements don't work, it's likely because the larger files are not located in the /data folder. To add them to the folder, do so by accessing the S3 bucket where they are located, using the statements below.

In [19]:
# My Bucket ARN Access Key, just in case
arn = 'arn:aws:s3:us-east-2:133716175259:accesspoint/recipe-book-GitHub-t2-exec-to-s3'

In [17]:
# try:
#     urllib.request.urlretrieve ('https://sagemaker-studio-t1ems8mtnoj.s3.us-east-2.amazonaws.com/RAW_interactions.csv', 'data/RAW_interactions.csv')
#     print('Success: downloaded RAW_interactions.csv.')
# except Exception as e:
#     print('Data load error: ',e)

# try:
#     idf = pd.read_csv('../data/RAW_interactions.csv',index_col=0)
#     print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

In [18]:
# try:
#     urllib.request.urlretrieve ('https://sagemaker-studio-t1ems8mtnoj.s3.us-east-2.amazonaws.com/RAW_recipes.csv', 'data/RAW_recipes.csv')
#     print('Success: downloaded RAW_recipes.csv.')
# except Exception as e:
#     print('Data load error: ',e)

# try:
#     rdf = pd.read_csv('../data/RAW_recipes.csv',index_col=0)
#     print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

Reference:  
>(*Both personalized and content-based recommendation systems*) make use of different similarity metrics to determine how "similar" items are to one another. The most common similarity metrics are [**Euclidean distance**](https://en.wikipedia.org/wiki/Euclidean_distance), [**cosine similarity**](https://en.wikipedia.org/wiki/Cosine_similarity), [**Pearson correlation**](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and the [**Jaccard index**](https://en.wikipedia.org/wiki/Jaccard_index) (useful with binary data). Each one of these distance metrics has its advantages and disadvantages depending on the type of ratings you are using and the characteristics of your data.$_1$  
  

> The second category covers the Model based approaches, which involve a step to reduce or compress the large but sparse user-item matrix. For understanding this step, a basic understanding of dimensionality reduction can be very helpful.

>It's worth pointing out that when SVD is calculated for recommendation systems, it is preferred to be done with a modified version called "Funk's SVD" that only takes into account the rated values, ignoring whatever items have not been rated by users. The algorithm is named after Simon Funk, who was part of the team who placed 3rd in the Netflix challenge with this innovative way of performing matrix decomposition. Read more about Funk's SVD implementation at his original blog post. There is no simple way to include for this fact with SciPy's implementation of svd(), but luckily the surprise library has Funk's version of SVD implemented to make our lives easier!
>
>Similar to other sklearn features, we can expedite the process of trying out different parameters by using an implementation of grid search. Let's make use of the grid search here to account for some different configurations of parameters within the SVD pipeline. This might take some time! You'll notice that the n_jobs is parameter set to -1, which ensures that all of the cores on your computer will be used to process fitting and evaluating all of these models. To help keep track of what is occurring here, take note of the different values. This code ended up taking over 16 minutes to complete even with parallelization in effect, so the optimal parameters are given to you for the SVD model below. Use them to train a model and let's see how well it performs. If you want the full grid search experience, feel free to uncomment the code and give it a go!

### Addtl. Package Installs & Import Statements

#### Additional Rec. System Packages:

- implicit
- lightfm
- pyspark.mlib.recommendation
- Amazon Personalize
- crab
- suggest

In [12]:
# %conda install --yes --prefix {sys.prefix} -c conda-forge implicit

In [13]:
# %conda install --yes --prefix {sys.prefix} -c conda-forge lightfm

#### Unused Packages Import Statements

In [14]:
## PYSPARK MODELING NOW IN SEPARATE DOCUMENT

# import pyspark
# import pyspark.sql.functions as F
# from pyspark.sql.types import ArrayType, IntegerType

## Further Reading

For those of you visiting this page who are interested in reading more about recommender systems, below are fantastic resources that I have collected.  
$_n$$_o$$_w$ &nbsp;$_I$ &nbsp;$_a$$_m$ &nbsp;$_t$$_h$$_e$ &nbsp;$_r$$_e$$_c$$_o$$_m$$_m$$_e$$_n$$_d$$_a$$_t$$_i$$_o$$_n$ &nbsp;$_s$$_y$$_s$$_t$$_e$$_m$$_!$

[*Mining Massive Datasets: Chapter 9*](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf), © Copyright Stanford University. Stanford, California 94305  
[Singular Value Decomposition (SVD) & Its Application In Recommender System](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/), Dr. Vaibhav Kumar

# Bibliography

1. [*Introduction to Recommender Systems* by Flatiron School](https://github.com/learn-co-curriculum/dsc-recommendation-system-introduction) is licensed under CC BY-NC-SA 4.0
>© 2018 Flatiron School, Inc.  
>© 2021 Flatiron School, LLC

# Notes

In [11]:
contents[0]

{'Key': 'RAW_interactions.csv',
 'LastModified': datetime.datetime(2021, 4, 29, 17, 32, 1, tzinfo=tzlocal()),
 'ETag': '"b76f667f757a6e0b6d3b08d23e0bc5e0-21"',
 'Size': 349436524,
 'StorageClass': 'STANDARD',
 'Owner': {'ID': 'ee74dd80de8b382fe7543b22cf2221ca7e68c76f28d55bec19b96bbe32873a4c'}}

In [12]:
contents[0]['Key']

'RAW_interactions.csv'

In [ ]:
# import boto3
# import sys

# if sys.version_info[0] < 3: 
#     from StringIO import StringIO # Python 2.x
# else:
#     from io import StringIO # Python 3.x

# # get your credentials from environment variables
# aws_id = os.environ['133716175259']
# aws_secret = os.environ['AWS_SECRET']

# client = boto3.client('s3', aws_access_key_id=aws_id,
#         aws_secret_access_key=aws_secret)

# bucket_name = 'my_bucket'

# object_key = 'my_file.csv'
# csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
# body = csv_obj['Body']
# csv_string = body.read().decode('utf-8')

# df = pd.read_csv(StringIO(csv_string))